# Zajęcia 1 (część 1)

Zakres:
* Zapoznanie się ze srodowiskiem pracy (Dataproc na Google Cloud Platform)
* Operacje na obiektowym systemie plików (GCS)
* Interaktywna analiza rozproszonych danych przy wykorzystaniu modułu Spark Core i Spark SQL
* Wprowadzenie modułu pandas
* Wizualizacje przy wykorzystaniu matplotlib oraz seaborn

Języki:
* bash, Python, SQL 

Krótka lista przydatnych poleceń bash: https://www.reddit.com/r/linux/comments/9rns12/some_linux_commands_cheatsheet/

### Jupyter Notebook

* Interaktywny notatnik dostepny poprzez przeglądarkę.
* Służy do wpisywania poleceń w wybranych jezykach programowania oraz opis w tzw jezyku markdown.
* Polecenia/opis wpisujemy do komórek. Komórki są wykonywane przez tzw. kernel (np Python w określonej wersji)
* Działa w tryb edycji komórek + tryb wykonywania poleceń. (przejście poprzez ESC i ENTER)
* Podstawowe skróty klawiaturowe: 
  * Esc/Enter
  * strzalki
  * ctrl-enter/shift-enter
  * a/b/d
  * y/m 
* Moze wykonywać polecenia powłoki (bash) !  %%bash
* Notatnik zapisywany jest w formacie ipynb (IPythonNoteBook)
* Kolejnosc uruchomienia komórek moze byc rozna.
* Mozliwosc zatrzymania kernela i uruchomienia od nowa. 

Polecamy film: https://www.youtube.com/watch?v=HW29067qVWk

Przykładowe komórki Pythonowe i Bashowe:

In [1]:
import os
user = os.environ.get('USER')
msg = f"Hello {user}!"
print (msg)

Hello root!


In [2]:
! pwd # pokaż aktualną scieżkę

/


In [3]:
%%bash
cd /root/ds-notebooks/session_1/
head 01_intro.ipynb
echo "--------------------"
tail 01_intro.ipynb

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Zajęcia 1 (część 1)\n",
    "\n",
    "Zakres:\n",
    "* Zapoznanie się ze srodowiskiem pracy (Dataproc na Google Cloud Platform)\n",
--------------------
   "version": "3.7.12"
  },
  "notebook_test": {
   "keytab_path": "/data/work/home/ds-lab-testuser1/ds-lab-testuser1.keytab",
   "user": "ds-lab-testuser1"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}


### Kopiowanie danych do swojego katalogu domowego

W katalogu `session1` znajduje sie plik który bedzie nam dzisiaj słuzyl do pracy.  
Dane pochodzą z https://gdac.broadinstitute.org/ i zawierają dane z badaniami nad mutacjami typu CNV w obrebie genu BRCA2.


In [4]:
%%bash
cd /root/ds-notebooks/session_1/
mkdir -p ~/data     # stworz katalog data (i posrednie)
cp brca.txt ~/data  # skopiuj plik z lokalizacji do bieżącej lokalizacji
ls -lah   ~/data       # wylistuj zawartość bieżącego katalogu

total 376K
drwxr-xr-x  2 root root 4.0K Dec 22 13:43 .
drwx------ 12 root root 4.0K Dec 22 13:43 ..
-rw-r--r--  1 root root 367K Dec 22 13:43 brca.txt


In [5]:
%%bash
head ~/data/brca.txt   # pokaż początek pliku
echo                   # pusta linia
tail ~/data/brca.txt   # pokaż koniec pliku 
echo
wc -l ~/data/brca.txt  # policz linie pliku

Sample	Chromosome	Start	End	Num_Probes	Segment_Mean
TCGA-A2-A0EU-10A-01D-A060-02	1	10209	2583075	NA	-0.210121164888764
TCGA-A2-A0EU-10A-01D-A060-02	1	2583076	249240606	NA	-0.101789555120838
TCGA-A2-A0EU-10A-01D-A060-02	2	10002	243189359	NA	-0.337963741969504
TCGA-A2-A0EU-10A-01D-A060-02	3	60175	162511435	NA	-0.0850647472883194
TCGA-A2-A0EU-10A-01D-A060-02	3	162511436	162626067	NA	0.42309157125509
TCGA-A2-A0EU-10A-01D-A060-02	3	162626068	197962416	NA	-0.0962225847981023
TCGA-A2-A0EU-10A-01D-A060-02	4	69223	191014397	NA	-0.0713350286479217
TCGA-A2-A0EU-10A-01D-A060-02	5	11770	180905246	NA	0.176064461471459
TCGA-A2-A0EU-10A-01D-A060-02	6	63815	171050932	NA	-0.0895478602860139

TCGA-BH-A0W5-01A-11D-A106-02	16	60001	35285758	NA	0.50100860888744
TCGA-BH-A0W5-01A-11D-A106-02	16	46499754	90294729	NA	-0.131450478937173
TCGA-BH-A0W5-01A-11D-A106-02	17	1	81195162	NA	-0.0320031758392927
TCGA-BH-A0W5-01A-11D-A106-02	18	10064	78017233	NA	-0.0384891054622891
TCGA-BH-A0W5-01A-11D-A106-02	19	70880	5911

## Środowisko Google Cloud Dataproc

### TODO Dataproc intro



## Wykorzystanie Google Cloud Storage

#### Projekt
* Wszystkie dane przynależą do konkretnego projektu.
* Do projektu mogą mieć dostęp użytkownicy. 
* Projekt ma zdefiniowane metody uwierzytelniające, rozliczenia, monitorowanie etc. 
#### Kubełek (bucket)
* Kubełek (bucket) to kontener na pliki/obiekty. 
* Nazwa Bucketu musi być unikalna w skali całej usługi u wszystkich użytkowników (!) 
* Kubełków nie można zagnieżdzać
* W kubełkach możemy tworzyć foldery i tam logicznie grupować pliki.
* Kubełek wraz z zawartością może zostać udostępniony publicznie.
* Kubełkowi nie można zmienić nazwy lub metadanych. Trzeba go usunąć i stworzyć ponownie.
#### Obiekt 
* obiekty przechowywane w kubełkach 
* obiekty mają zawartość oraz metadane
* obiekty są niemodyfikowalne 

Do operacji na Google Storage można wykorzystać narzędzie `gsutil`: 

##### Operacje na kubełkach
* listowanie kubełków (buckets) - `ls`
* tworzenie nowego kubełka - `mb`
* usuwania kubełka - `rm`
* listowanie zawartości kubełków - `ls` 
* udostępnianie - `iam`

##### Operacje na obiektach
* dodawania pliku do kubełka - `cp`
* kopiowanie - `cp`
* usuniecie z kubełka - `cp`
* pobranie informacji o obiekcie - `stat`

#### Operacje na kubełkach

In [1]:
! gsutil ls

gs://dataproc-staging-europe-west1-328044151187-f0lebcfu/
gs://dataproc-temp-europe-west1-328044151187-qwiuvc1i/
gs://ds-2024l-7001-notebook-data/
gs://ds-2024l-7001-state/


In [12]:
! echo $USER  # wyswietlenie nazwy uzytkownika

root


In [13]:
# TODO Please update user id
%env USER_ID=7701
%env SEMESTER=2024l

env: USER_ID=7701
env: SEMESTER=2024l


In [15]:
! gsutil mb gs://big-data-lab-$SEMESTER-$USER_ID

Creating gs://big-data-lab-2024l-7701/...
ServiceException: 409 A Cloud Storage bucket named 'big-data-lab-2024l-7701' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [16]:
! gsutil ls gs://big-data-lab-$SEMESTER-$USER_ID

In [20]:
! gsutil du -s gs://big-data-lab-$SEMESTER-$USER_ID # ile zajmuje przestrzeni?

14           gs://big-data-lab-2024l-7701


In [10]:
! gsutil du -L -b gs://big-data-lab-$SEMESTER-$USER_ID # listowanie zawartości 

CommandException: Incorrect option(s) specified. Usage:

  gsutil du url...

For additional help run:
  gsutil help du


#### Operacje na zawartości kubełków

In [17]:
! gsutil ls -r gs://big-data-lab-$SEMESTER-$USER_ID  # listowanie zawartosci kubełka

In [18]:
! gsutil cp ~/ds-notebooks/README.md gs://big-data-lab-$SEMESTER-$USER_ID  # upload obiektu do kubełka

Copying file:///root/ds-notebooks/README.md [Content-Type=text/markdown]...
- [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       


In [19]:
! gsutil ls -r gs://big-data-lab-$SEMESTER-$USER_ID  # listowanie zawartosci kubełka

gs://big-data-lab-2024l-7701/README.md


In [21]:
! gsutil iam get gs://big-data-lab-$SEMESTER-$USER_ID

{
  "bindings": [
    {
      "members": [
        "projectEditor:ds-2024l-7001",
        "projectOwner:ds-2024l-7001"
      ],
      "role": "roles/storage.legacyBucketOwner"
    },
    {
      "members": [
        "projectViewer:ds-2024l-7001"
      ],
      "role": "roles/storage.legacyBucketReader"
    }
  ],
  "etag": "CAE="
}


In [22]:
! gsutil stat gs://big-data-lab-$SEMESTER-$USER_ID/README.md # metadane obiektu w kubełku

gs://big-data-lab-2024l-7701/README.md:
    Creation time:          Sun, 22 Dec 2024 14:11:41 GMT
    Update time:            Sun, 22 Dec 2024 14:11:41 GMT
    Storage class:          STANDARD
    Content-Length:         14
    Content-Type:           text/markdown
    Hash (crc32c):          e+o8YQ==
    Hash (md5):             MxzbFtnEwBhVlFmJHMLZeQ==
    ETag:                   CKLup6zHu4oDEAE=
    Generation:             1734876701783842
    Metageneration:         1


#### Przeniesienie do docelowego kubełka

## <span style='background:yellow'> ZADANIE 1 </span>
Wgraj plik brca.txt do kubełka `gs://ds-$SEMESTER-$USER_ID-notebook-data` do katalogu `data`.

In [34]:
! gsutil mb gs://ds-$SEMESTER-$USER_ID-notebook-data
! gsutil cp ~/data/brca.txt gs://ds-$SEMESTER-$USER_ID-notebook-data/data/brca.txt
! gsutil cat gs://ds-$SEMESTER-$USER_ID-notebook-data/data/brca.txt

Creating gs://ds-2024l-7701-notebook-data/...
Copying file:///root/data/brca.txt [Content-Type=text/plain]...
- [1 files][366.8 KiB/366.8 KiB]                                                
Operation completed over 1 objects/366.8 KiB.                                    
Sample	Chromosome	Start	End	Num_Probes	Segment_Mean
TCGA-A2-A0EU-10A-01D-A060-02	1	10209	2583075	NA	-0.210121164888764
TCGA-A2-A0EU-10A-01D-A060-02	1	2583076	249240606	NA	-0.101789555120838
TCGA-A2-A0EU-10A-01D-A060-02	2	10002	243189359	NA	-0.337963741969504
TCGA-A2-A0EU-10A-01D-A060-02	3	60175	162511435	NA	-0.0850647472883194
TCGA-A2-A0EU-10A-01D-A060-02	3	162511436	162626067	NA	0.42309157125509
TCGA-A2-A0EU-10A-01D-A060-02	3	162626068	197962416	NA	-0.0962225847981023
TCGA-A2-A0EU-10A-01D-A060-02	4	69223	191014397	NA	-0.0713350286479217
TCGA-A2-A0EU-10A-01D-A060-02	5	11770	180905246	NA	0.176064461471459
TCGA-A2-A0EU-10A-01D-A060-02	6	63815	171050932	NA	-0.0895478602860139
TCGA-A2-A0EU-10A-01D-A060-02	7	10128	15912864

In [35]:
! gsutil ls -la gs://ds-$SEMESTER-$USER_ID-notebook-data

                                 gs://ds-2024l-7701-notebook-data/data/


## <span style='background:yellow'> ZADANIE 2 </span>
a) Utwórz plik zawierający aktualną datę i godzinę.  
b) Wrzuć plik do kubełka `gs://big-data-lab-$SEMESTER-$USER_ID`.

### Pobranie pliku z powrotem na lokalny dysk

In [ ]:
! gsutil cp gs://ds-$SEMESTER-$USER_ID-notebook-data/data/brca.txt .  # gdzie sie zapisal ten plik?

## Apache Spark

* Apache Spark - platforma ogólnego zastosowania, open source, do przetwarzania duzych zbiorów danych.
* Posiada API dla języków programowania: Scala, Python i R. 
* Przetwarzanie w Spark przetwarzanie jest wykonywane w większości  wprost w pamięci operacyjnej.
* Przeznaczenie: do uruchamiania  aplikacji i skryptów z wykorzystaniem uczenia maszynowego lub interaktywnych kwerend.
* Spark ten wspiera SQL (typ DataFrames), przetwarzanie strumieniowe oraz przetwarzanie grafów.
* Integracja z lokalną pamięci masową, rozproszonymi lub obiektowymi systemu plików.
* Spark można uruchamić na pojedynczej maszynie na środowisku klastrowym, lub w chmurze. 

Zakres na dzisiejsze laboratorium:
* stworzenie sesji Spark
* zaczytanie danych z pliku tekstowego
* kwerendy na danych

Będziemy korzystać z pliku ktory zapisalismy na GCS w pierwszej części ćwiczenia.

In [36]:
! gsutil ls -r gs://ds-$SEMESTER-$USER_ID-notebook-data

gs://ds-2024l-7701-notebook-data/data/:
gs://ds-2024l-7701-notebook-data/data/brca.txt


### Przygotowanie sesji Sparkowej


In [37]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.config("spark.executor.instances", "1") \
.config("spark.executor.memory", "1g") \
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/22 14:27:35 INFO SparkEnv: Registering MapOutputTracker
24/12/22 14:27:35 INFO SparkEnv: Registering BlockManagerMaster
24/12/22 14:27:35 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/12/22 14:27:36 INFO SparkEnv: Registering OutputCommitCoordinator


In [109]:
spark.stop()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.config("spark.executor.instances", "1") \
.config("spark.executor.memory", "1g") \
.getOrCreate()

### Ręczne utworzenie zbioru danych

In [38]:
# stworzenie rozproszonego DF na podstawie podanej listy krotek 
df_users = spark.createDataFrame(
    [(123, 'Jan Kowalski', 'jan@kowalski.pl'), 
     (124, 'Anna Nowak', 'anna@nowak.pl'), 
     (125, 'Janusz Kowalski', 'janusz@kowalski.pl'), 
     (126, 'Anita Nowak', 'anita@nowak.pl')])

In [39]:
df_users.show()

+---+---------------+------------------+
| _1|             _2|                _3|
+---+---------------+------------------+
|123|   Jan Kowalski|   jan@kowalski.pl|
|124|     Anna Nowak|     anna@nowak.pl|
|125|Janusz Kowalski|janusz@kowalski.pl|
|126|    Anita Nowak|    anita@nowak.pl|
+---+---------------+------------------+



In [40]:
df_users.printSchema()

root
 |-- _1: long (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)



In [41]:
df_users = spark.createDataFrame(
    [(123, 'Jan Kowalski', 'jan@kowalski.pl'), 
     (124, 'Anna Nowak', 'anna@nowak.pl'), 
     (125, 'Janusz Kowalski', 'janusz@kowalski.pl'), 
     (126, 'Anita Nowak', 'anita@nowak.pl')],
    schema='id int not null, name string, email string')

In [43]:
df_users.show()

+---+---------------+------------------+
| id|           name|             email|
+---+---------------+------------------+
|123|   Jan Kowalski|   jan@kowalski.pl|
|124|     Anna Nowak|     anna@nowak.pl|
|125|Janusz Kowalski|janusz@kowalski.pl|
|126|    Anita Nowak|    anita@nowak.pl|
+---+---------------+------------------+



In [44]:
df_users.printSchema()

root
 |-- id: integer (nullable = false)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)




### Wczytanie danych
Zazwyczaj dane mamy przechowane na zewnątrz naszego notatnika (np. w pliku) i należy je odczytać.

In [45]:
# TODO Please update user id
semester = '2024l'
user_id = '7701'
input_path = f'gs://ds-{semester}-{user_id}-notebook-data/data/brca.txt'

In [46]:
df = spark.read.load(input_path, format="csv", sep="\t", inferSchema="true", header="true")

### Charakterystyka danych

In [47]:
type (df)                 # jaki jest typ danych

pyspark.sql.dataframe.DataFrame

In [48]:
df.explain()              # fizyczny plan wykonania

== Physical Plan ==
FileScan csv [Sample#68,Chromosome#69,Start#70,End#71,Num_Probes#72,Segment_Mean#73] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[gs://ds-2024l-7701-notebook-data/data/brca.txt], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Sample:string,Chromosome:int,Start:int,End:int,Num_Probes:string,Segment_Mean:double>




In [49]:
df.explain(True)          # logiczny i fizyczny plan wykonania

== Parsed Logical Plan ==
Relation [Sample#68,Chromosome#69,Start#70,End#71,Num_Probes#72,Segment_Mean#73] csv

== Analyzed Logical Plan ==
Sample: string, Chromosome: int, Start: int, End: int, Num_Probes: string, Segment_Mean: double
Relation [Sample#68,Chromosome#69,Start#70,End#71,Num_Probes#72,Segment_Mean#73] csv

== Optimized Logical Plan ==
Relation [Sample#68,Chromosome#69,Start#70,End#71,Num_Probes#72,Segment_Mean#73] csv

== Physical Plan ==
FileScan csv [Sample#68,Chromosome#69,Start#70,End#71,Num_Probes#72,Segment_Mean#73] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[gs://ds-2024l-7701-notebook-data/data/brca.txt], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Sample:string,Chromosome:int,Start:int,End:int,Num_Probes:string,Segment_Mean:double>



In [50]:
df.rdd.getNumPartitions() # liczba partycji (bloków danych)

1

In [51]:
df.printSchema()          # schemat danych

root
 |-- Sample: string (nullable = true)
 |-- Chromosome: integer (nullable = true)
 |-- Start: integer (nullable = true)
 |-- End: integer (nullable = true)
 |-- Num_Probes: string (nullable = true)
 |-- Segment_Mean: double (nullable = true)



In [52]:
df.count()                # wymiary (liczba wierszy)

5298

In [53]:
len(df.columns)           # wymiary (liczba kolumn)

6

In [55]:
df.describe().show()      # pokaż podsumowanie danych w tabeli

+-------+--------------------+------------------+-------------------+-------------------+----------+------------------+
|summary|              Sample|        Chromosome|              Start|                End|Num_Probes|      Segment_Mean|
+-------+--------------------+------------------+-------------------+-------------------+----------+------------------+
|  count|                5298|              5298|               5298|               5298|      5298|              5298|
|   mean|                NULL|11.312948282370705|5.464405806266516E7|7.557261170932427E7|      NULL|0.2103169573704153|
| stddev|                NULL| 6.548797844563064|5.276515600857485E7|5.816178219098644E7|      NULL|0.8318029444477086|
|    min|TCGA-A2-A0EU-01A-...|                 1|                  1|              99263|        NA| -3.49124544908987|
|    max|TCGA-BH-A0W5-10A-...|                24|          249152065|          249240606|        NA|  4.25258455583762|
+-------+--------------------+----------

In [56]:
df.describe("Chromosome").show() # pokaż podsumowanie danych w kolumnie 

+-------+------------------+
|summary|        Chromosome|
+-------+------------------+
|  count|              5298|
|   mean|11.312948282370705|
| stddev| 6.548797844563064|
|    min|                 1|
|    max|                24|
+-------+------------------+



### Odczyt danych
Z wykorzystaniem API DataFrame

In [57]:
df.show()                    # pokaż 20 wierszy danych 

+--------------------+----------+---------+---------+----------+-------------------+
|              Sample|Chromosome|    Start|      End|Num_Probes|       Segment_Mean|
+--------------------+----------+---------+---------+----------+-------------------+
|TCGA-A2-A0EU-10A-...|         1|    10209|  2583075|        NA| -0.210121164888764|
|TCGA-A2-A0EU-10A-...|         1|  2583076|249240606|        NA| -0.101789555120838|
|TCGA-A2-A0EU-10A-...|         2|    10002|243189359|        NA| -0.337963741969504|
|TCGA-A2-A0EU-10A-...|         3|    60175|162511435|        NA|-0.0850647472883194|
|TCGA-A2-A0EU-10A-...|         3|162511436|162626067|        NA|   0.42309157125509|
|TCGA-A2-A0EU-10A-...|         3|162626068|197962416|        NA|-0.0962225847981023|
|TCGA-A2-A0EU-10A-...|         4|    69223|191014397|        NA|-0.0713350286479217|
|TCGA-A2-A0EU-10A-...|         5|    11770|180905246|        NA|  0.176064461471459|
|TCGA-A2-A0EU-10A-...|         6|    63815|171050932|        NA|-

In [58]:
df.show(10, truncate=False) # pokaż 10 wierszy, nie skracaj danych

+----------------------------+----------+---------+---------+----------+-------------------+
|Sample                      |Chromosome|Start    |End      |Num_Probes|Segment_Mean       |
+----------------------------+----------+---------+---------+----------+-------------------+
|TCGA-A2-A0EU-10A-01D-A060-02|1         |10209    |2583075  |NA        |-0.210121164888764 |
|TCGA-A2-A0EU-10A-01D-A060-02|1         |2583076  |249240606|NA        |-0.101789555120838 |
|TCGA-A2-A0EU-10A-01D-A060-02|2         |10002    |243189359|NA        |-0.337963741969504 |
|TCGA-A2-A0EU-10A-01D-A060-02|3         |60175    |162511435|NA        |-0.0850647472883194|
|TCGA-A2-A0EU-10A-01D-A060-02|3         |162511436|162626067|NA        |0.42309157125509   |
|TCGA-A2-A0EU-10A-01D-A060-02|3         |162626068|197962416|NA        |-0.0962225847981023|
|TCGA-A2-A0EU-10A-01D-A060-02|4         |69223    |191014397|NA        |-0.0713350286479217|
|TCGA-A2-A0EU-10A-01D-A060-02|5         |11770    |180905246|NA       

In [59]:
df.select("Sample").show()  # pokaż tylko kolumne sample

+--------------------+
|              Sample|
+--------------------+
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
+--------------------+
only showing top 20 rows



In [60]:
df.select("Chromosome", "Start", "End").show() # pokaż kolumny chromosome, start i end

+----------+---------+---------+
|Chromosome|    Start|      End|
+----------+---------+---------+
|         1|    10209|  2583075|
|         1|  2583076|249240606|
|         2|    10002|243189359|
|         3|    60175|162511435|
|         3|162511436|162626067|
|         3|162626068|197962416|
|         4|    69223|191014397|
|         5|    11770|180905246|
|         6|    63815|171050932|
|         7|    10128|159128640|
|         8|    23380|  1238776|
|         8|  1238777|  1246507|
|         8|  1246508|  1316313|
|         8|  1316314|  3129168|
|         8|  3129169| 36339136|
|         8| 36339137| 39701358|
|         8| 39701359| 40086720|
|         8| 40086721| 48901482|
|         8| 48901483| 48911575|
|         8| 48911576|146303846|
+----------+---------+---------+
only showing top 20 rows



In [61]:
df_chrom = df.select(df.Chromosome).distinct() # stworz nowy DF zawierajacy tylko unikalne wartosci chromosomow

In [62]:
df_chrom.count()                     # policz wiersze w nowym DF

24

In [63]:
df.filter("Chromosome > 21").show() # pokaz dane spelniajace warunek (przy uzyciu filter)

+--------------------+----------+--------+---------+----------+--------------------+
|              Sample|Chromosome|   Start|      End|Num_Probes|        Segment_Mean|
+--------------------+----------+--------+---------+----------+--------------------+
|TCGA-A2-A0EU-10A-...|        22|16972573| 51244552|        NA|  -0.459539615359244|
|TCGA-A2-A0EU-10A-...|        23| 2699504|154930285|        NA|   0.194146928345973|
|TCGA-A2-A0EU-01A-...|        22|16972573| 51244552|        NA|  -0.459539615359244|
|TCGA-A2-A0EU-01A-...|        23| 2699504|154930285|        NA|   0.194146928345973|
|TCGA-A7-A0D9-10A-...|        22|16051207| 51244552|        NA|  -0.226265745441125|
|TCGA-A7-A0D9-10A-...|        23| 2699504|154930285|        NA|  -0.186598214609633|
|TCGA-A7-A0D9-01A-...|        22|16051207| 51244552|        NA|  -0.226265745441125|
|TCGA-A7-A0D9-01A-...|        23| 2699504|154930285|        NA|  -0.186598214609633|
|TCGA-AO-A0JF-10A-...|        22|16051206| 51244552|        NA| 0

In [ ]:
df.where("Chromosome > 21").show() # pokaz dane spelniajace warunek (przy uzyciu when)

In [ ]:
df.filter("Chromosome > 21").explain()  # pokaz plan wykonania

In [ ]:
df.filter("Chromosome > 21 and Segment_Mean > 0").show() # pokaz dane spelniajace warunki

### Grupowanie i funkcje agregujące

In [64]:
from pyspark.sql.functions import *

In [65]:
df.groupBy("Chromosome").count().show()   # dokonaj grupowania po chromosomie i policz rekordy w grupie

+----------+-----+
|Chromosome|count|
+----------+-----+
|        12|  202|
|         1|  288|
|         6|  360|
|         3|  192|
|         4|  254|
|         8|  556|
|        11|  318|
|        19|  136|
|        23|  104|
|        21|   96|
|        14|  128|
|        16|  222|
|        20|  388|
|         5|  154|
|        15|  168|
|        24|    8|
|         2|  286|
|        18|  238|
|        13|  120|
|        17|  440|
+----------+-----+
only showing top 20 rows



In [66]:
df.groupBy("Chromosome").avg("Segment_Mean").show() # dokonaj grupowania po chromosomie i policz srednia wartosc segment_mean w grupie

+----------+--------------------+
|Chromosome|   avg(Segment_Mean)|
+----------+--------------------+
|        12|  0.2077776566255012|
|         1|  0.2749020151479202|
|         6| 0.09061564099997978|
|         3|-0.02618245312138...|
|         4|-0.01256776781881...|
|         8|  0.6859064011278554|
|        11| 0.03372826519655162|
|        19| 0.08623909396694052|
|        23|-0.07294761544611646|
|        21|-0.06047204925394...|
|        14| 0.28058573260745223|
|        16|  0.3809825220709315|
|        20|  0.7508020639591368|
|         5|-0.04673099374977487|
|        15|  0.6376181402358202|
|        24| 0.07405671005723104|
|         2|-0.11265343721466738|
|        18| 0.09047098750564904|
|        13|-0.25463230407325826|
|        17| 0.24462914215524065|
+----------+--------------------+
only showing top 20 rows



In [67]:
# pogrupuj dane wzgledem probki i chromosomu, policz rekordy w grupie
df.groupBy("Sample","Chromosome").count().orderBy(asc("Sample")).show()

+--------------------+----------+-----+
|              Sample|Chromosome|count|
+--------------------+----------+-----+
|TCGA-A2-A0EU-01A-...|        19|    1|
|TCGA-A2-A0EU-01A-...|        17|    1|
|TCGA-A2-A0EU-01A-...|        18|    1|
|TCGA-A2-A0EU-01A-...|         3|    3|
|TCGA-A2-A0EU-01A-...|        14|    1|
|TCGA-A2-A0EU-01A-...|        13|    1|
|TCGA-A2-A0EU-01A-...|        12|    1|
|TCGA-A2-A0EU-01A-...|         4|    1|
|TCGA-A2-A0EU-01A-...|        23|    1|
|TCGA-A2-A0EU-01A-...|        22|    1|
|TCGA-A2-A0EU-01A-...|        21|    1|
|TCGA-A2-A0EU-01A-...|        20|    1|
|TCGA-A2-A0EU-01A-...|         7|    1|
|TCGA-A2-A0EU-01A-...|        16|    6|
|TCGA-A2-A0EU-01A-...|         6|    1|
|TCGA-A2-A0EU-01A-...|        11|    1|
|TCGA-A2-A0EU-01A-...|        15|    1|
|TCGA-A2-A0EU-01A-...|         8|   10|
|TCGA-A2-A0EU-01A-...|         5|    1|
|TCGA-A2-A0EU-01A-...|         2|    1|
+--------------------+----------+-----+
only showing top 20 rows



### Kolumny wyliczane

In [68]:
df.withColumn ("Length", col("End") - col("Start")).show() # dodaj kolumne dlugosc jako end - start

+--------------------+----------+---------+---------+----------+-------------------+---------+
|              Sample|Chromosome|    Start|      End|Num_Probes|       Segment_Mean|   Length|
+--------------------+----------+---------+---------+----------+-------------------+---------+
|TCGA-A2-A0EU-10A-...|         1|    10209|  2583075|        NA| -0.210121164888764|  2572866|
|TCGA-A2-A0EU-10A-...|         1|  2583076|249240606|        NA| -0.101789555120838|246657530|
|TCGA-A2-A0EU-10A-...|         2|    10002|243189359|        NA| -0.337963741969504|243179357|
|TCGA-A2-A0EU-10A-...|         3|    60175|162511435|        NA|-0.0850647472883194|162451260|
|TCGA-A2-A0EU-10A-...|         3|162511436|162626067|        NA|   0.42309157125509|   114631|
|TCGA-A2-A0EU-10A-...|         3|162626068|197962416|        NA|-0.0962225847981023| 35336348|
|TCGA-A2-A0EU-10A-...|         4|    69223|191014397|        NA|-0.0713350286479217|190945174|
|TCGA-A2-A0EU-10A-...|         5|    11770|1809052

In [69]:
df.withColumn ("Material", lit("DNA")).show()     # dodaj kolumne o stalej wartosci 'DNA'

+--------------------+----------+---------+---------+----------+-------------------+--------+
|              Sample|Chromosome|    Start|      End|Num_Probes|       Segment_Mean|Material|
+--------------------+----------+---------+---------+----------+-------------------+--------+
|TCGA-A2-A0EU-10A-...|         1|    10209|  2583075|        NA| -0.210121164888764|     DNA|
|TCGA-A2-A0EU-10A-...|         1|  2583076|249240606|        NA| -0.101789555120838|     DNA|
|TCGA-A2-A0EU-10A-...|         2|    10002|243189359|        NA| -0.337963741969504|     DNA|
|TCGA-A2-A0EU-10A-...|         3|    60175|162511435|        NA|-0.0850647472883194|     DNA|
|TCGA-A2-A0EU-10A-...|         3|162511436|162626067|        NA|   0.42309157125509|     DNA|
|TCGA-A2-A0EU-10A-...|         3|162626068|197962416|        NA|-0.0962225847981023|     DNA|
|TCGA-A2-A0EU-10A-...|         4|    69223|191014397|        NA|-0.0713350286479217|     DNA|
|TCGA-A2-A0EU-10A-...|         5|    11770|180905246|       

In [70]:
df.withColumn ("Chromosome2", concat(lit("chr"), col("Chromosome"))).show() # dodaj kolumne z konkatencja

+--------------------+----------+---------+---------+----------+-------------------+-----------+
|              Sample|Chromosome|    Start|      End|Num_Probes|       Segment_Mean|Chromosome2|
+--------------------+----------+---------+---------+----------+-------------------+-----------+
|TCGA-A2-A0EU-10A-...|         1|    10209|  2583075|        NA| -0.210121164888764|       chr1|
|TCGA-A2-A0EU-10A-...|         1|  2583076|249240606|        NA| -0.101789555120838|       chr1|
|TCGA-A2-A0EU-10A-...|         2|    10002|243189359|        NA| -0.337963741969504|       chr2|
|TCGA-A2-A0EU-10A-...|         3|    60175|162511435|        NA|-0.0850647472883194|       chr3|
|TCGA-A2-A0EU-10A-...|         3|162511436|162626067|        NA|   0.42309157125509|       chr3|
|TCGA-A2-A0EU-10A-...|         3|162626068|197962416|        NA|-0.0962225847981023|       chr3|
|TCGA-A2-A0EU-10A-...|         4|    69223|191014397|        NA|-0.0713350286479217|       chr4|
|TCGA-A2-A0EU-10A-...|        

In [71]:
df.drop("Chromosome").show()  # usun kolumne Chromosome. 
# Czy DF została pozbawiona kolumny na trwale?

+--------------------+---------+---------+----------+-------------------+
|              Sample|    Start|      End|Num_Probes|       Segment_Mean|
+--------------------+---------+---------+----------+-------------------+
|TCGA-A2-A0EU-10A-...|    10209|  2583075|        NA| -0.210121164888764|
|TCGA-A2-A0EU-10A-...|  2583076|249240606|        NA| -0.101789555120838|
|TCGA-A2-A0EU-10A-...|    10002|243189359|        NA| -0.337963741969504|
|TCGA-A2-A0EU-10A-...|    60175|162511435|        NA|-0.0850647472883194|
|TCGA-A2-A0EU-10A-...|162511436|162626067|        NA|   0.42309157125509|
|TCGA-A2-A0EU-10A-...|162626068|197962416|        NA|-0.0962225847981023|
|TCGA-A2-A0EU-10A-...|    69223|191014397|        NA|-0.0713350286479217|
|TCGA-A2-A0EU-10A-...|    11770|180905246|        NA|  0.176064461471459|
|TCGA-A2-A0EU-10A-...|    63815|171050932|        NA|-0.0895478602860139|
|TCGA-A2-A0EU-10A-...|    10128|159128640|        NA|-0.0402976850808555|
|TCGA-A2-A0EU-10A-...|    23380|  1238

### Zapis wyników

In [72]:
df2 = df.withColumn ("Chromosome2", concat(lit("chr"), col("Chromosome")))  # dodaj kolumne z konkatenacja i zapisz do nowego DF

In [73]:
output_path = f'gs://ds-{semester}-{user_id}-notebook-data/data/brca2.txt'
df2.write.format("csv").mode("overwrite").save(output_path)

In [74]:
! gsutil ls gs://ds-$SEMESTER-$USER_ID-notebook-data/data/

gs://ds-2024l-7701-notebook-data/data/brca.txt
gs://ds-2024l-7701-notebook-data/data/brca2.txt/


In [75]:
!gsutil cat gs://ds-$SEMESTER-$USER_ID-notebook-data/data/brca.txt | head

Sample	Chromosome	Start	End	Num_Probes	Segment_Mean
TCGA-A2-A0EU-10A-01D-A060-02	1	10209	2583075	NA	-0.210121164888764
TCGA-A2-A0EU-10A-01D-A060-02	1	2583076	249240606	NA	-0.101789555120838
TCGA-A2-A0EU-10A-01D-A060-02	2	10002	243189359	NA	-0.337963741969504
TCGA-A2-A0EU-10A-01D-A060-02	3	60175	162511435	NA	-0.0850647472883194
TCGA-A2-A0EU-10A-01D-A060-02	3	162511436	162626067	NA	0.42309157125509
TCGA-A2-A0EU-10A-01D-A060-02	3	162626068	197962416	NA	-0.0962225847981023
TCGA-A2-A0EU-10A-01D-A060-02	4	69223	191014397	NA	-0.0713350286479217
TCGA-A2-A0EU-10A-01D-A060-02	5	11770	180905246	NA	0.176064461471459
TCGA-A2-A0EU-10A-01D-A060-02	6	63815	171050932	NA	-0.0895478602860139


#### Wazne!
Na koniec pracy przy tym notatniku (po wykonaniu poniższego polecenia) koniecznie zatrzymaj sesję Sparkową:

In [ ]:
spark.stop()

## <span style='background:yellow'> ZADANIE 3</span>

a) Ile jest unikalnych danych próbek w tym zbiorze danych?

b) Pokaż nazwę próbki, numer chromosomu, początek i koniec segmentu dla segmentów występujących na chromosomie 21,22,23 i mających startową pozycję większą niż 10000000. 

c) Ile występuje wierszy dla każdej z próbki? Pokaż wynik z pełną nazwą próbki. Zapisz wynik do pliku na GCS

In [103]:
df.select(df.Sample) \
  .distinct() \
  .count()

38

In [104]:
df.filter("Chromosome in ('21', '22', '23') and Start > 10000000") \
  .select("Sample", "Chromosome", "Start", "End") \
  .show()

+--------------------+----------+---------+---------+
|              Sample|Chromosome|    Start|      End|
+--------------------+----------+---------+---------+
|TCGA-A2-A0EU-10A-...|        22| 16972573| 51244552|
|TCGA-A2-A0EU-01A-...|        22| 16972573| 51244552|
|TCGA-A7-A0D9-10A-...|        22| 16051207| 51244552|
|TCGA-A7-A0D9-01A-...|        22| 16051207| 51244552|
|TCGA-AO-A0JF-10A-...|        22| 16051206| 51244552|
|TCGA-AO-A0JF-01A-...|        22| 16051206| 51244552|
|TCGA-AO-A0JJ-10A-...|        22| 16051206| 51244552|
|TCGA-AO-A0JJ-10A-...|        23| 79266839|116067549|
|TCGA-AO-A0JJ-01A-...|        22| 16051206| 51244552|
|TCGA-AO-A0JJ-01A-...|        23| 79266839|116067549|
|TCGA-AO-A0JL-10A-...|        22| 16051206| 47154099|
|TCGA-AO-A0JL-10A-...|        22| 47154100| 51061870|
|TCGA-AO-A0JL-10A-...|        22| 51061871| 51244552|
|TCGA-AO-A0JL-10A-...|        23| 30938120| 31584900|
|TCGA-AO-A0JL-10A-...|        23| 31584901| 71333082|
|TCGA-AO-A0JL-10A-...|      

In [105]:
df3 = df.groupBy("Sample") \
        .count()

In [106]:
df3.show(truncate=False)

+----------------------------+-----+
|Sample                      |count|
+----------------------------+-----+
|TCGA-AR-A0TU-01A-31D-A106-02|263  |
|TCGA-BH-A0W5-10A-01D-A106-02|26   |
|TCGA-B6-A0RI-01A-11D-A060-02|54   |
|TCGA-BH-A0H6-01A-21D-A060-02|35   |
|TCGA-BH-A0HK-10A-01D-A060-02|68   |
|TCGA-B6-A0RE-01A-11D-A060-02|186  |
|TCGA-B6-A0X4-10A-01D-A106-02|28   |
|TCGA-B6-A0RG-10A-01D-A060-02|104  |
|TCGA-AO-A0JL-01A-11D-A060-02|316  |
|TCGA-A7-A0D9-01A-31D-A060-02|182  |
|TCGA-A7-A0D9-10A-01D-A060-02|182  |
|TCGA-AO-A0JJ-10A-01D-A060-02|33   |
|TCGA-BH-A0W5-01A-11D-A106-02|26   |
|TCGA-BH-A0BM-10A-01D-A060-02|33   |
|TCGA-A2-A0EU-10A-01D-A060-02|42   |
|TCGA-AO-A0JF-10A-01D-A060-02|28   |
|TCGA-BH-A0GY-01A-11D-A060-02|124  |
|TCGA-B6-A0RI-10A-01D-A060-02|54   |
|TCGA-AO-A0JF-01A-11D-A060-02|28   |
|TCGA-BH-A0HB-01A-11D-A060-02|474  |
+----------------------------+-----+
only showing top 20 rows



In [107]:
output_path = f'gs://ds-{semester}-{user_id}-notebook-data/data/brca_sample_counts.txt'
df3.write.format("csv").mode("overwrite").save(output_path)